In [2]:
"""import blockulib.models as blom
import torch

model = blom.ConvModel()
input_tensor = torch.randn(1, 1, 9, 9)
output = model(input_tensor)
print("New model sample output: ", output)

PATH = "models/conv_model.pth"
torch.save(model.state_dict(), PATH) """

'import blockulib.models as blom\nimport torch\n\nmodel = blom.ConvModel()\ninput_tensor = torch.randn(1, 1, 9, 9)\noutput = model(input_tensor)\nprint("New model sample output: ", output)\n\nPATH = "models/conv_model.pth"\ntorch.save(model.state_dict(), PATH) '

In [3]:
import torch
import torch.nn.functional as F

def sample_from_logits(logits, temperature = 1.0, top_k: int = None):
    logits = logits / temperature

    if top_k is not None and top_k < logits.size(-1):
        top_vals, _ = torch.topk(logits, k=top_k)
        threshold = top_vals[-1]
        logits[logits < threshold] = float('-inf')

    probs = F.softmax(logits, dim=-1)
    sampled_index = torch.multinomial(probs, num_samples=1)
    return sampled_index.item()

In [4]:
def logits_to_choices(logits, index, num_games, temperature = 1.0, top_k: int = None):
    range_left = [None for i in range(num_games)]
    range_right = [None for i in range(num_games)]
        
    for i in range(index.shape[0]):
        ind = int(index[i].item())
        if range_left[ind] is None:
            range_left[ind] = i
        range_right[ind] = i+1
            
    choices = []
    for left, right in zip(range_left, range_right):
        if left is not None and right is not None:
            choices.append(sample_from_logits(logits[left:right], temperature = temperature, top_k = top_k))
        else:
            choices.append(None)
    return choices

In [5]:
"""logits = torch.tensor([2.1, 3.5, 2.2, 2.12,  1.5, 9.0])
index = torch.tensor([0, 0, 0, 0, 1, 1])

for i in range(20):
    choices = logits_to_choices(logits, index, top_k = 3)
    print(choices)"""

'logits = torch.tensor([2.1, 3.5, 2.2, 2.12,  1.5, 9.0])\nindex = torch.tensor([0, 0, 0, 0, 1, 1])\n\nfor i in range(20):\n    choices = logits_to_choices(logits, index, top_k = 3)\n    print(choices)'

In [6]:
import blockulib
import blockulib.models as blom
import torch

class PlayingLoop():
    
    def __init__(self, model_path = "models/conv_model.pth", architecture = blom.ConvModel):
        self.model = architecture()
        state_dict = torch.load(model_path)
        self.model.load_state_dict(state_dict)
        self.generator = blockulib.BlockGenerator()
        self.model.eval()
        
    def __call__(self, num_games = 1, batch_size = 2048):
        pos_list = [[torch.zeros(9, 9)] for i in range(num_games)]
        state = [True for i in range(num_games)]
        active_games = num_games
        move = 0
        
        while (active_games > 0):
            move += 1
            if (move == 1 or move%5 == 0):
                print("Entering move phase: ", move)
            new_index = []
            for i in range(num_games):
                if state[i]:
                    new_index.append(i)
            boards = [pos_list[new_index[i]][-1].clone() for i in range(active_games)]
            
            pos, ind = blockulib.possible_moves(boards, self.generator)
            logits = self.get_model_pred(pos, batch_size = batch_size).squeeze(1)
            decisions = logits_to_choices(logits, ind, active_games)
            
            if (move == 1 or move%5 == 0):
                print("Possible moves count: ", ind.shape[0])
            
            for i in range(active_games):
                if (decisions[i] is None):
                    state[new_index[i]] = False
                    active_games -= 1
                else:
                    pos_list[new_index[i]].append(pos[decisions[i]])
                    
        print("ended after ", move, " moves")            
        return pos_list
                    
    def get_model_pred(self, data, batch_size, device = None):
        if (data.shape[0] == 0):
            return torch.tensor([[]])
        if device is None:
            device = ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        predictions = []
        
        with torch.no_grad():
            for i in range(0, data.shape[0], batch_size):
                batch = data[i:i+batch_size].to(device)
                batch = batch.unsqueeze(1)
                output = self.model(batch)
                predictions.append(output.cpu())
            
        return torch.cat(predictions)

In [7]:
play = PlayingLoop()

In [8]:
import time
start_time = time.time()

wynik = play(num_games = 1000)
drugi = play(num_games = 1000)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Entering move phase:  1
Possible moves count:  59311
Entering move phase:  5
Possible moves count:  42328
Entering move phase:  10
Possible moves count:  16318
Entering move phase:  15
Possible moves count:  7432
Entering move phase:  20
Possible moves count:  2401
Entering move phase:  25
Possible moves count:  782
Entering move phase:  30
Possible moves count:  60
Entering move phase:  35
Possible moves count:  52
ended after  39  moves
Entering move phase:  1
Possible moves count:  58363
Entering move phase:  5
Possible moves count:  32971
Entering move phase:  10
Possible moves count:  19064
Entering move phase:  15
Possible moves count:  4961
Entering move phase:  20
Possible moves count:  912
Entering move phase:  25
Possible moves count:  1157
Entering move phase:  30
Possible moves count:  90
Entering move phase:  35
Possible moves count:  35
Entering move phase:  40
Possible moves count:  6
ended after  43  moves
Elapsed time: 4.072512 seconds


In [9]:
print(len(wynik))

1000


In [10]:
cases = 0
for i in range(1000):
    cases += len(wynik[i])
print(cases, "cases int total")

18408 cases int total
